# Recuperation des données Espagne WEB 

In [ ]:
# Importation des bibliothèques nécessaires
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime


In [ ]:

# Étape 1: Récupération de la page web
def get_webpage_content(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"Erreur lors de la récupération de la page: {e}")
        return None

# Étape 2: Extraction des liens spécifiques
def extract_review_links(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Recherche de tous les liens
    review_links = []
    
    # Utilisation d'une expression régulière pour trouver les liens spécifiques
    pattern = r'https?://[^\s<>"]+?/spain/[^\s<>"]+?reviews\.csv'
    links = re.findall(pattern, html_content)
    
    # Nettoie et ajoute les liens valides
    for link in links:
        if link.endswith('reviews.csv') and 'spain' in link.lower():
            review_links.append(link)
    
    # Supprime les doublons
    review_links = list(set(review_links))
    return review_links

# Étape 3: Création du DataFrame avec informations supplémentaires
def create_dataframe(links):
    data = []
    for link in links:
        # Extraction des informations de l'URL
        parts = link.split('/')
        ville = parts[-4] if len(parts) >= 4 else ''
        date = parts[-3] if len(parts) >= 3 else ''
        
        data.append({
            'url': link,
            'ville': ville,
            'date': date
        })
    
    # Création du DataFrame
    df = pd.DataFrame(data)
    return df

# Étape 4: Exécution du code
def main():
    url = "https://insideairbnb.com/get-the-data/"
    print("Récupération des données depuis le site...")
    html_content = get_webpage_content(url)
    
    if html_content:
        review_links = extract_review_links(html_content)
        
        # Création du DataFrame
        df = create_dataframe(review_links)
        
        # Affichage des informations
        print(f"\nNombre de liens reviews.csv trouvés: {len(review_links)}")
        print("\nAperçu du DataFrame:")
        print(df)
        
        return df  # Retourne le DataFrame pour utilisation ultérieure

# Exécution du programme
if __name__ == "__main__":
    df_results = main()

# Partie DATA LAKE

In [5]:
# 1. Import des bibliothèques nécessaires
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [6]:

# 2. Chargement des variables d'environnement
load_dotenv()

# 2. Configuration des informations d'authentification
# Configuration Key Vault
key_vault_url = os.getenv('KEY_VAULT_URL')
secret_name = os.getenv('WRITE_SECRET_NAME')

# Configuration Azure AD
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('WRITE_CLIENT_ID')

# Configuration Storage
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
container_name = os.getenv('CONTAINER_NAME')


In [7]:

# 3. Récupération du secret depuis Key Vault
def get_secret_from_keyvault():
    try:
        # Utilisation de DefaultAzureCredential pour l'authentification
        credential = DefaultAzureCredential()
        
        # Création du client Key Vault
        secret_client = SecretClient(vault_url=key_vault_url, credential=credential)
        
        # Récupération du secret
        secret = secret_client.get_secret(secret_name)
        return secret.value
        
    except Exception as e:
        print(f"Erreur lors de la récupération du secret: {str(e)}")
        return None

In [8]:
# 4. Création des credentials


def create_credentials():
    client_secret = get_secret_from_keyvault()
    if client_secret:
        return ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )
    return None

In [9]:
# 5. Fonction pour transférer un seul fichier CSV
def transfer_single_csv(url, file_system_client, ville):
    try:
        # Création du nom de fichier de destination
        file_name = f"{ville}_reviews.csv"
        destination_path = f"airbnb_data/spain/{ville}/{file_name}"
        
        print(f"\n📥 Début du transfert pour {ville}...")
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        # Création du fichier dans le Data Lake
        file_client = file_system_client.create_file(destination_path)
        
        current_position = 0
        for chunk in response.iter_content(chunk_size=4*1024*1024):
            if chunk:
                file_client.append_data(data=chunk, offset=current_position)
                current_position += len(chunk)
                
                if total_size:
                    progress = (current_position / total_size) * 100
                    print(f"\r{ville}: {progress:.2f}% ({current_position/(1024*1024):.2f} Mo)", end='')
        
        file_client.flush_data(current_position)
        print(f"\n✅ {ville} transféré avec succès!")
        return True, ville, current_position
        
    except Exception as e:
        print(f"\n❌ Erreur lors du transfert de {ville}: {str(e)}")
        return False, ville, 0

In [10]:
# 5. Fonction pour transférer un seul fichier CSV
def transfer_single_csv(url, file_system_client, ville):
    try:
        # Création du nom de fichier de destination
        file_name = f"{ville}_reviews.csv"
        destination_path = f"airbnb_data/spain/{ville}/{file_name}"
        
        print(f"\n📥 Début du transfert pour {ville}...")
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        # Création du fichier dans le Data Lake
        file_client = file_system_client.create_file(destination_path)
        
        current_position = 0
        for chunk in response.iter_content(chunk_size=4*1024*1024):
            if chunk:
                file_client.append_data(data=chunk, offset=current_position)
                current_position += len(chunk)
                
                if total_size:
                    progress = (current_position / total_size) * 100
                    print(f"\r{ville}: {progress:.2f}% ({current_position/(1024*1024):.2f} Mo)", end='')
        
        file_client.flush_data(current_position)
        print(f"\n✅ {ville} transféré avec succès!")
        return True, ville, current_position
        
    except Exception as e:
        print(f"\n❌ Erreur lors du transfert de {ville}: {str(e)}")
        return False, ville, 0

In [11]:
# 6. Fonction principale pour traiter le DataFrame
def process_dataframe_urls(df, max_workers=3):
    try:
        print(f"🚀 Démarrage du transfert pour {len(df)} fichiers...")
        
        # Configuration du client Data Lake
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")
        
        account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        service_client = DataLakeServiceClient(account_url, credential=credential)
        file_system_client = service_client.get_file_system_client(container_name)
        
        successful_transfers = 0
        total_size = 0
        
        # Traitement parallèle des fichiers
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_url = {
                executor.submit(
                    transfer_single_csv,
                    row['url'],
                    file_system_client,
                    row['ville']
                ): row['ville']
                for _, row in df.iterrows()
            }
            
            for future in as_completed(future_to_url):
                success, ville, size = future.result()
                if success:
                    successful_transfers += 1
                    total_size += size
        
        print(f"\n📊 Résumé du transfert:")
        print(f"- Fichiers transférés avec succès: {successful_transfers}/{len(df)}")
        print(f"- Taille totale transférée: {total_size/(1024*1024*1024):.2f} Go")
        
    except Exception as e:
        print(f"❌ Erreur générale: {str(e)}")

In [12]:
# 7. Utilisation du code
if __name__ == "__main__":

    
    MAX_WORKERS = 3  # Nombre de téléchargements simultanés
    
    print("🎯 Configuration:")
    print(f"- Nombre de fichiers à traiter: {len(df_results)}")
    print(f"- {MAX_WORKERS} téléchargements simultanés")
    
    # Exécution du transfert
    process_dataframe_urls(df_results, MAX_WORKERS)

🎯 Configuration:
- Nombre de fichiers à traiter: 18
- 3 téléchargements simultanés
🚀 Démarrage du transfert pour 18 fichiers...

📥 Début du transfert pour girona...

📥 Début du transfert pour mallorca...

📥 Début du transfert pour sevilla...

❌ Erreur lors du transfert de sevilla: 403 Client Error: Forbidden for url: https://data.insideairbnb.com/spain/andaluc%C3%83%C2%ADa/sevilla/2024-06-30/data/reviews.csv

📥 Début du transfert pour valencia...
girona: 100.00% (7.72 Mo)
✅ girona transféré avec succès!

📥 Début du transfert pour madrid...
madrid: 100.00% (26.23 Mo))
✅ madrid transféré avec succès!

📥 Début du transfert pour menorca...
menorca: 100.00% (21.89 Mo)
✅ menorca transféré avec succès!

📥 Début du transfert pour barcelona...
valencia: 100.00% (121.21 Mo)
✅ valencia transféré avec succès!

📥 Début du transfert pour sevilla...

❌ Erreur lors du transfert de sevilla: 403 Client Error: Forbidden for url: https://data.insideairbnb.com/spain/andaluc%C3%83%C2%ADa/sevilla/2024-06-30/